# BlueTrails AI - Water Quality Prediction Inference

**Purpose**: Generate future water quality predictions and store them in the database

**Process**:
1. Load best models for each parameter (hybrid V2/V4 approach)
2. Generate future dates for prediction
3. Prepare features for all sites
4. Run predictions using best models
5. Bulk insert predictions to database

**Best Models** (based on V4 weighted scoring analysis):
- **CHL_A**: V2 MLP (R²=0.21, MAE=3.04)
- **Turbidity**: V4 Random Forest (R²=0.44, MAE=2.62)
- **DO_mg_l**: V4 Ridge Regression (R²=0.38, MAE=1.32)
- **N_TOTAL**: V2 MLP (R²=0.30, MAE=194.22)
- **Temperature**: V4 Random Forest (R²=0.90, MAE=0.97)

**Date Range**: Configurable start_date and end_date parameters

## 1. Setup & Import Libraries

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pickle
from datetime import datetime, timedelta
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')
import uuid

# PyTorch for V2 models
import torch
import torch.nn as nn

# Scikit-learn
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Supabase
from supabase import create_client, Client

# Set seeds
np.random.seed(34328637)
torch.manual_seed(34328637)

print(f"✓ Libraries loaded successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

✓ Libraries loaded successfully!
PyTorch version: 2.8.0+cpu
Device: CPU


## 2. Configuration Parameters

In [2]:
# ============================================================================
# CONFIGURATION - Modify these parameters as needed
# ============================================================================

# Prediction date range
START_DATE = '2025-10-03'  # Start date for predictions
END_DATE = '2025-10-31'    # End date for predictions

# Prediction frequency (days between predictions)
FREQUENCY_DAYS = 1

# Model version identifier
MODEL_VERSION = 'hybrid_v2_v4_2025'

# Best models configuration (based on weighted scoring analysis)
BEST_MODELS = {
    'chl_a': {
        'version': 'v2',
        'model_file': '../model/internal/v2/chl_a_model.pth',
        'model_name': 'V2_MLP',
        'param_db_name': 'CHL_A',
        'r2': 0.2103,
    },
    'turbidity': {
        'version': 'v4',
        'model_file': '../model/internal/v4/turbidity_classical_ml.pkl',
        'model_name': 'V4_RandomForest',
        'param_db_name': 'Turb',
        'r2': 0.4395,
    },
    'do_mg_l': {
        'version': 'v4',
        'model_file': '../model/internal/v4/do_mg_l_classical_ml.pkl',
        'model_name': 'V4_Ridge',
        'param_db_name': 'DO_mg',
        'r2': 0.3838,
    },
    'n_total': {
        'version': 'v2',
        'model_file': '../model/internal/v2/n_total_model.pth',
        'model_name': 'V2_MLP',
        'param_db_name': 'N_TOTAL',
        'r2': 0.2992,
    },
    'temperature': {
        'version': 'v4',
        'model_file': '../model/internal/v4/temperature_classical_ml.pkl',
        'model_name': 'V4_RandomForest',
        'param_db_name': 'Temperature',
        'r2': 0.8982,
    },
}

print("\n" + "="*70)
print("INFERENCE CONFIGURATION")
print("="*70)
print(f"Date Range: {START_DATE} to {END_DATE}")
print(f"Frequency: Every {FREQUENCY_DAYS} days")
print(f"Model Version: {MODEL_VERSION}")
print(f"\nBest Models:")
for param, config in BEST_MODELS.items():
    print(f"  {param:15} - {config['model_name']:20} (R²={config['r2']:.4f})")
print("="*70)


INFERENCE CONFIGURATION
Date Range: 2025-10-03 to 2025-10-31
Frequency: Every 1 days
Model Version: hybrid_v2_v4_2025

Best Models:
  chl_a           - V2_MLP               (R²=0.2103)
  turbidity       - V4_RandomForest      (R²=0.4395)
  do_mg_l         - V4_Ridge             (R²=0.3838)
  n_total         - V2_MLP               (R²=0.2992)
  temperature     - V4_RandomForest      (R²=0.8982)


## 3. Connect to Supabase

In [3]:
load_dotenv('../.env')

SUPABASE_URL = os.getenv('SUPABASE_URL')
SUPABASE_KEY = os.getenv('SUPABASE_KEY')

if not SUPABASE_URL or not SUPABASE_KEY:
    raise ValueError("Please set SUPABASE_URL and SUPABASE_KEY in .env file")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
print("✓ Connected to Supabase")

✓ Connected to Supabase


## 4. Load Historical Data for Feature Engineering

In [4]:
# Fetch historical data
response = supabase.table('v_epa_measurements_wide').select('*').execute()
df_historical = pd.DataFrame(response.data)

# Fetch sites
response_sites = supabase.table('v_epa_sites').select('*').execute()
df_sites = pd.DataFrame(response_sites.data)

# Fetch param IDs for database insertion
response_params = supabase.table('epa_param').select('id, name').execute()
df_params = pd.DataFrame(response_params.data)
param_id_map = dict(zip(df_params['name'], df_params['id']))

# Fetch type ID for 'surface'
response_types = supabase.table('epa_type').select('id, name').execute()
df_types = pd.DataFrame(response_types.data)
surface_type_id = df_types[df_types['name'] == 'surface']['id'].values[0]

print(f"✓ Loaded {len(df_historical)} historical measurements")
print(f"✓ Found {len(df_sites)} monitoring sites")
print(f"✓ Parameter ID mapping: {param_id_map}")
print(f"✓ Surface type ID: {surface_type_id}")

✓ Loaded 5853 historical measurements
✓ Found 21 monitoring sites
✓ Parameter ID mapping: {'CHL_A': 1, 'Secchi_depth_m': 2, 'DO_mg': 3, 'FL': 4, 'PAR': 5, 'Sal': 6, 'Temp': 7, 'Turb': 8, 'DO_sat': 9, 'Temperature': 10, 'pH': 11, 'TSS': 12, 'N_NH3': 13, 'N_NO2': 14, 'N_NO3': 15, 'N_NOX': 16, 'N_TOTAL': 17, 'P_PO4': 18, 'P_TOTAL': 19, 'SI': 20}
✓ Surface type ID: 1


## 5. Prepare Historical Features

In [5]:
# Filter to surface measurements
df = df_historical[df_historical['measurement_type'] == 'surface'].copy()

# Merge with site metadata
df = df.merge(df_sites[['site_id', 'latitude', 'longitude', 'water_body_name']], 
              on='site_id', how='left', suffixes=('', '_site'))

df['latitude'] = df['latitude_site'].fillna(df['latitude'])
df['longitude'] = df['longitude_site'].fillna(df['longitude'])
df = df.drop(['latitude_site', 'longitude_site'], axis=1, errors='ignore')

# Convert date
df['date'] = pd.to_datetime(df['date'])

# Define target parameters (same as training)
df['temperature'] = df['temp_sensor'].fillna(df['temp_lab'])
TARGET_PARAMS = ['chl_a', 'turbidity', 'do_mg_l', 'n_total', 'temperature']

print(f"✓ Prepared {len(df)} surface measurements")
print(f"✓ Date range: {df['date'].min()} to {df['date'].max()}")

✓ Prepared 4492 surface measurements
✓ Date range: 1984-07-19 00:00:00 to 2025-06-20 00:00:00


## 6. Feature Engineering (Same as Training)

In [6]:
# Encode categorical features (fit on historical data)
site_encoder = LabelEncoder()
df['site_id_encoded'] = site_encoder.fit_transform(df['site_id'])

water_body_encoder = LabelEncoder()
df['water_body_encoded'] = water_body_encoder.fit_transform(df['water_body_name'].fillna('Unknown'))

# Sort by site and date
df = df.sort_values(['site_id', 'date']).reset_index(drop=True)

# Calculate site statistics (needed for future predictions)
site_stats = df.groupby('site_id')[TARGET_PARAMS].agg(['mean', 'std']).reset_index()
site_stats.columns = ['site_id'] + [f'{param}_{stat}' for param in TARGET_PARAMS for stat in ['mean', 'std']]

# Get last known values per site (for lag features)
last_values = df.sort_values('date').groupby('site_id')[TARGET_PARAMS].last().reset_index()
last_values.columns = ['site_id'] + [f'{param}_last' for param in TARGET_PARAMS]

print("✓ Feature engineering complete")
print(f"✓ Encoded {df['site_id'].nunique()} sites and {df['water_body_name'].nunique()} water bodies")

✓ Feature engineering complete
✓ Encoded 21 sites and 3 water bodies


## 7. Generate Future Dates for Prediction

In [7]:
# Generate prediction dates
start_date = pd.to_datetime(START_DATE)
end_date = pd.to_datetime(END_DATE)

prediction_dates = pd.date_range(start=start_date, end=end_date, freq=f'{FREQUENCY_DAYS}D')

print(f"\n{'='*70}")
print(f"PREDICTION DATES")
print(f"{'='*70}")
print(f"Total dates: {len(prediction_dates)}")
print(f"Dates: {list(prediction_dates.strftime('%Y-%m-%d'))}")
print(f"{'='*70}")


PREDICTION DATES
Total dates: 29
Dates: ['2025-10-03', '2025-10-04', '2025-10-05', '2025-10-06', '2025-10-07', '2025-10-08', '2025-10-09', '2025-10-10', '2025-10-11', '2025-10-12', '2025-10-13', '2025-10-14', '2025-10-15', '2025-10-16', '2025-10-17', '2025-10-18', '2025-10-19', '2025-10-20', '2025-10-21', '2025-10-22', '2025-10-23', '2025-10-24', '2025-10-25', '2025-10-26', '2025-10-27', '2025-10-28', '2025-10-29', '2025-10-30', '2025-10-31']


## 8. Create Prediction Dataset

In [8]:
# Create cartesian product of sites × dates
sites_list = df_sites['site_id'].unique()
prediction_data = []

for site_id in sites_list:
    for pred_date in prediction_dates:
        prediction_data.append({
            'site_id': site_id,
            'date': pred_date,
        })

df_predict = pd.DataFrame(prediction_data)

# Merge with site metadata
df_predict = df_predict.merge(df_sites[['site_id', 'latitude', 'longitude', 'water_body_name']], 
                               on='site_id', how='left')

# Merge with site statistics
df_predict = df_predict.merge(site_stats, on='site_id', how='left')

# Merge with last known values (for lag features)
df_predict = df_predict.merge(last_values, on='site_id', how='left')

# Create temporal features
df_predict['year'] = df_predict['date'].dt.year
df_predict['month'] = df_predict['date'].dt.month
df_predict['day_of_year'] = df_predict['date'].dt.dayofyear
df_predict['season'] = df_predict['month'].apply(lambda m: 
    1 if m in [12, 1, 2] else 2 if m in [3, 4, 5] else 3 if m in [6, 7, 8] else 4
)

# Cyclic encoding
df_predict['month_sin'] = np.sin(2 * np.pi * df_predict['month'] / 12)
df_predict['month_cos'] = np.cos(2 * np.pi * df_predict['month'] / 12)
df_predict['day_sin'] = np.sin(2 * np.pi * df_predict['day_of_year'] / 365)
df_predict['day_cos'] = np.cos(2 * np.pi * df_predict['day_of_year'] / 365)

# Encode categorical features
df_predict['site_id_encoded'] = site_encoder.transform(df_predict['site_id'])
df_predict['water_body_encoded'] = water_body_encoder.transform(df_predict['water_body_name'].fillna('Unknown'))

# Create lag features from last known values
for param in TARGET_PARAMS:
    df_predict[f'{param}_lag1'] = df_predict[f'{param}_last']

print(f"\n✓ Created prediction dataset with {len(df_predict)} rows")
print(f"  Sites: {len(sites_list)}")
print(f"  Dates: {len(prediction_dates)}")
print(f"  Total predictions: {len(sites_list) * len(prediction_dates) * len(TARGET_PARAMS)} (across all parameters)")


✓ Created prediction dataset with 609 rows
  Sites: 21
  Dates: 29
  Total predictions: 3045 (across all parameters)


## 9. Define Feature List (Same as Training)

In [9]:
# Core features
CORE_FEATURES = [
    'site_id_encoded', 'water_body_encoded', 'latitude', 'longitude',
    'year', 'month', 'season',
    'month_sin', 'month_cos', 'day_sin', 'day_cos',
]

# Site statistics
STAT_FEATURES = [col for col in df_predict.columns if any(stat in col for stat in ['_mean', '_std'])]

# Lag features
LAG_FEATURES = [f'{param}_lag1' for param in TARGET_PARAMS]

# All features
ALL_FEATURES = CORE_FEATURES + STAT_FEATURES + LAG_FEATURES

print(f"Total features: {len(ALL_FEATURES)}")
print(f"  Core: {len(CORE_FEATURES)}")
print(f"  Site Stats: {len(STAT_FEATURES)}")
print(f"  Lag: {len(LAG_FEATURES)}")

# Impute missing values (same as training)
core_imputer = SimpleImputer(strategy='median')
df_predict[CORE_FEATURES] = core_imputer.fit_transform(df_predict[CORE_FEATURES])

stat_imputer = SimpleImputer(strategy='median')
df_predict[STAT_FEATURES] = stat_imputer.fit_transform(df_predict[STAT_FEATURES])

lag_imputer = SimpleImputer(strategy='median')
df_predict[LAG_FEATURES] = lag_imputer.fit_transform(df_predict[LAG_FEATURES])

print("\n✓ All features imputed")

Total features: 26
  Core: 11
  Site Stats: 10
  Lag: 5

✓ All features imputed


## 10. Define V2 MLP Architecture (for loading)

In [10]:
class MLPRegressor(nn.Module):
    """
    Multi-Layer Perceptron for regression (V2 architecture).
    """
    def __init__(self, input_dim, hidden_dim1=128, hidden_dim2=64, dropout=0.3):
        super(MLPRegressor, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1),
            nn.BatchNorm1d(hidden_dim1),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.BatchNorm1d(hidden_dim2),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            nn.Linear(hidden_dim2, 1)
        )
    
    def forward(self, x):
        return self.network(x).squeeze()

print("✓ V2 MLP architecture defined")

✓ V2 MLP architecture defined


## 11. Load Models and Run Predictions

In [12]:
# Storage for predictions
predictions = {}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"\n{'='*70}")
print("LOADING MODELS AND RUNNING PREDICTIONS")
print(f"{'='*70}")

for param, config in BEST_MODELS.items():
    print(f"\n{param.upper()}:")
    print(f"  Model: {config['model_name']}")
    print(f"  Version: {config['version']}")
    print(f"  File: {config['model_file']}")
    
    if config['version'] == 'v2':
        # Load V2 PyTorch model
        model_artifact = torch.load(config['model_file'], map_location=device, weights_only=False)
        
        # Extract components
        scaler_X = model_artifact['scaler_X']
        scaler_y = model_artifact['scaler_y']
        
        # Initialize model
        input_dim = len(ALL_FEATURES)
        model = MLPRegressor(input_dim, hidden_dim1=128, hidden_dim2=64, dropout=0.3).to(device)
        model.load_state_dict(model_artifact['model_state_dict'])
        model.eval()
        
        # Prepare data
        X = df_predict[ALL_FEATURES].values
        X_scaled = scaler_X.transform(X)
        X_tensor = torch.FloatTensor(X_scaled).to(device)
        
        # Predict
        with torch.no_grad():
            y_pred_scaled = model(X_tensor).cpu().numpy()
        
        # Inverse transform
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
        
    elif config['version'] == 'v4':
        # Load V4 classical ML model
        with open(config['model_file'], 'rb') as f:
            model_artifact = pickle.load(f)
        
        model = model_artifact['model']
        scaler_X = model_artifact['scaler_X']
        poly = model_artifact.get('poly', None)
        
        # Prepare data
        X = df_predict[ALL_FEATURES].values
        X_scaled = scaler_X.transform(X)
        
        # Apply polynomial features if needed
        if poly is not None:
            X_scaled = poly.transform(X_scaled)
        
        # Predict
        y_pred = model.predict(X_scaled)
    
    # Store predictions
    predictions[param] = y_pred
    
    print(f"  ✓ Generated {len(y_pred)} predictions")
    print(f"    Min: {y_pred.min():.4f}")
    print(f"    Max: {y_pred.max():.4f}")
    print(f"    Mean: {y_pred.mean():.4f}")

print(f"\n{'='*70}")
print("✓ ALL PREDICTIONS GENERATED SUCCESSFULLY!")
print(f"{'='*70}")


LOADING MODELS AND RUNNING PREDICTIONS

CHL_A:
  Model: V2_MLP
  Version: v2
  File: ../model/internal/v2/chl_a_model.pth
  ✓ Generated 609 predictions
    Min: 0.2974
    Max: 21.8591
    Mean: 3.3688

TURBIDITY:
  Model: V4_RandomForest
  Version: v4
  File: ../model/internal/v4/turbidity_classical_ml.pkl
  ✓ Generated 609 predictions
    Min: 0.4551
    Max: 26.9733
    Mean: 4.0541

DO_MG_L:
  Model: V4_Ridge
  Version: v4
  File: ../model/internal/v4/do_mg_l_classical_ml.pkl
  ✓ Generated 609 predictions
    Min: 7.4266
    Max: 50.0391
    Mean: 10.4020

N_TOTAL:
  Model: V2_MLP
  Version: v2
  File: ../model/internal/v2/n_total_model.pth
  ✓ Generated 609 predictions
    Min: 101.1961
    Max: 832.9276
    Mean: 305.9594

TEMPERATURE:
  Model: V4_RandomForest
  Version: v4
  File: ../model/internal/v4/temperature_classical_ml.pkl
  ✓ Generated 609 predictions
    Min: 14.2728
    Max: 17.5036
    Mean: 16.1415

✓ ALL PREDICTIONS GENERATED SUCCESSFULLY!


## 12. Prepare Data for Database Insertion

In [18]:
# Generate prediction run ID
prediction_run_id = str(uuid.uuid4())

# Create prediction metadata
prediction_metadata = {
    'prediction_run_id': prediction_run_id,
    'model_version': MODEL_VERSION,
    'start_date': START_DATE,
    'end_date': END_DATE,
    'site_count': len(sites_list),
    'prediction_count': len(df_predict) * len(TARGET_PARAMS),
    'model_info': {
        'best_models': {k: {'model_name': v['model_name'], 'r2': v['r2']} for k, v in BEST_MODELS.items()},
        'frequency_days': FREQUENCY_DAYS,
        'features': ALL_FEATURES,
    }
}

print(f"Prediction Run ID: {prediction_run_id}")
print(f"Total predictions: {len(df_predict) * len(TARGET_PARAMS)}")

Prediction Run ID: 619d8e03-212c-4145-a6bd-71439df9f28f
Total predictions: 3045


In [19]:
# Prepare prediction records for bulk insert
prediction_records = []

for idx, row in df_predict.iterrows():
    site_id = row['site_id']
    date = row['date'].strftime('%Y-%m-%d')
    
    for param, y_pred_value in predictions.items():
        param_config = BEST_MODELS[param]
        param_db_name = param_config['param_db_name']
        param_id = param_id_map[param_db_name]
        
        # Calculate confidence score from R²
        # Use R² as proxy for confidence (0 to 1 scale)
        # Negative R² becomes 0 confidence
        r2 = param_config['r2']
        confidence_score = max(0.0, min(1.0, r2))  # Clamp to [0, 1]
        
        prediction_records.append({
            'prediction_run_id': prediction_run_id,
            'site_id': site_id,
            'date': date,
            'type_id': int(surface_type_id),
            'param_id': int(param_id),
            'param_value': float(y_pred_value[idx]),
            'confidence_score': float(confidence_score),
            'model_name': param_config['model_name'],
        })

print(f"\n✓ Prepared {len(prediction_records)} prediction records for insertion")
print(f"\nSample records:")
for i in range(min(5, len(prediction_records))):
    rec = prediction_records[i]
    print(f"  {rec['site_id']} | {rec['date']} | Param ID {rec['param_id']} | "
          f"Value: {rec['param_value']:.4f} | Confidence: {rec['confidence_score']:.4f} | "
          f"Model: {rec['model_name']}")


✓ Prepared 3045 prediction records for insertion

Sample records:
  2316 | 2025-10-03 | Param ID 1 | Value: 4.2296 | Confidence: 0.2103 | Model: V2_MLP
  2316 | 2025-10-03 | Param ID 8 | Value: 2.5955 | Confidence: 0.4395 | Model: V4_RandomForest
  2316 | 2025-10-03 | Param ID 3 | Value: 9.8049 | Confidence: 0.3838 | Model: V4_Ridge
  2316 | 2025-10-03 | Param ID 17 | Value: 370.6689 | Confidence: 0.2992 | Model: V2_MLP
  2316 | 2025-10-03 | Param ID 10 | Value: 16.1066 | Confidence: 0.8982 | Model: V4_RandomForest


## 13. Insert Predictions into Database

In [20]:
print(f"\n{'='*70}")
print("INSERTING PREDICTIONS INTO DATABASE")
print(f"{'='*70}")

# Step 1: Insert prediction metadata
print(f"\n1. Inserting prediction metadata...")
try:
    response = supabase.table('epa_prediction_metadata').insert(prediction_metadata).execute()
    print(f"   ✓ Metadata inserted successfully")
except Exception as e:
    print(f"   ✗ Error inserting metadata: {e}")
    raise

# Step 2: Bulk insert predictions (in batches for safety)
print(f"\n2. Inserting {len(prediction_records)} prediction records...")

BATCH_SIZE = 1000  # Insert 1000 records at a time
total_batches = (len(prediction_records) + BATCH_SIZE - 1) // BATCH_SIZE

for batch_idx in range(total_batches):
    start_idx = batch_idx * BATCH_SIZE
    end_idx = min((batch_idx + 1) * BATCH_SIZE, len(prediction_records))
    batch = prediction_records[start_idx:end_idx]
    
    try:
        response = supabase.table('epa_data_prediction').insert(batch).execute()
        print(f"   ✓ Batch {batch_idx+1}/{total_batches} inserted ({len(batch)} records)")
    except Exception as e:
        print(f"   ✗ Error inserting batch {batch_idx+1}: {e}")
        raise

print(f"\n{'='*70}")
print("✓ ALL PREDICTIONS INSERTED SUCCESSFULLY!")
print(f"{'='*70}")
print(f"\nPrediction Run ID: {prediction_run_id}")
print(f"Total records inserted: {len(prediction_records)}")
print(f"Date range: {START_DATE} to {END_DATE}")
print(f"Sites: {len(sites_list)}")
print(f"Parameters: {len(TARGET_PARAMS)}")


INSERTING PREDICTIONS INTO DATABASE

1. Inserting prediction metadata...
   ✓ Metadata inserted successfully

2. Inserting 3045 prediction records...
   ✓ Batch 1/4 inserted (1000 records)
   ✓ Batch 2/4 inserted (1000 records)
   ✓ Batch 3/4 inserted (1000 records)
   ✓ Batch 4/4 inserted (45 records)

✓ ALL PREDICTIONS INSERTED SUCCESSFULLY!

Prediction Run ID: 619d8e03-212c-4145-a6bd-71439df9f28f
Total records inserted: 3045
Date range: 2025-10-03 to 2025-10-31
Sites: 21
Parameters: 5


## 14. Verify Insertion

In [21]:
# Query back the inserted predictions
print(f"\nVerifying insertion...")

response = supabase.table('v_epa_predictions').select('*').eq('prediction_run_id', prediction_run_id).limit(10).execute()
df_verify = pd.DataFrame(response.data)

print(f"\nSample predictions from database:")
print(df_verify[['site_id', 'date', 'param_name', 'param_value', 'confidence_score', 'model_name']].head(10))

# Get count
response_count = supabase.table('epa_data_prediction').select('id', count='exact').eq('prediction_run_id', prediction_run_id).execute()
db_count = response_count.count

print(f"\n✓ Verified {db_count} records in database")
print(f"  Expected: {len(prediction_records)}")
print(f"  Match: {db_count == len(prediction_records)}")


Verifying insertion...

Sample predictions from database:
  site_id        date   param_name  param_value  confidence_score  \
0     716  2025-10-31        CHL_A     0.885489            0.2103   
1     716  2025-10-31        DO_mg     7.592948            0.3838   
2     716  2025-10-31      N_TOTAL   151.277298            0.2992   
3     716  2025-10-31  Temperature    16.712745            0.8982   
4     716  2025-10-31         Turb     1.477570            0.4395   
5    1229  2025-10-31        CHL_A     0.465133            0.2103   
6    1229  2025-10-31        DO_mg     7.801047            0.3838   
7    1229  2025-10-31      N_TOTAL   135.583740            0.2992   
8    1229  2025-10-31  Temperature    15.946181            0.8982   
9    1229  2025-10-31         Turb     0.600315            0.4395   

        model_name  
0           V2_MLP  
1         V4_Ridge  
2           V2_MLP  
3  V4_RandomForest  
4  V4_RandomForest  
5           V2_MLP  
6         V4_Ridge  
7           V

## 15. Summary Statistics

In [22]:
# Create summary DataFrame
summary_data = []

for param, config in BEST_MODELS.items():
    y_pred = predictions[param]
    
    summary_data.append({
        'Parameter': param.upper(),
        'Model': config['model_name'],
        'R² Score': config['r2'],
        'Predictions': len(y_pred),
        'Min': y_pred.min(),
        'Max': y_pred.max(),
        'Mean': y_pred.mean(),
        'Std': y_pred.std(),
    })

df_summary = pd.DataFrame(summary_data)

print(f"\n{'='*100}")
print("PREDICTION SUMMARY")
print(f"{'='*100}")
print(df_summary.to_string(index=False))
print(f"{'='*100}")

print(f"\n✅ Inference complete!")
print(f"\nNext steps:")
print(f"  1. Query predictions via: SELECT * FROM v_epa_predictions WHERE prediction_run_id = '{prediction_run_id}'")
print(f"  2. Wide format view: SELECT * FROM v_epa_measurements_wide_prediction WHERE prediction_run_id = '{prediction_run_id}'")
print(f"  3. Integrate with backend API to serve predictions to frontend")


PREDICTION SUMMARY
  Parameter           Model  R² Score  Predictions        Min        Max       Mean        Std
      CHL_A          V2_MLP    0.2103          609   0.297403  21.859091   3.368788   5.040491
  TURBIDITY V4_RandomForest    0.4395          609   0.455079  26.973292   4.054108   5.644768
    DO_MG_L        V4_Ridge    0.3838          609   7.426567  50.039085  10.401955   8.880754
    N_TOTAL          V2_MLP    0.2992          609 101.196121 832.927551 305.959412 211.579010
TEMPERATURE V4_RandomForest    0.8982          609  14.272797  17.503591  16.141479   0.697973

✅ Inference complete!

Next steps:
  1. Query predictions via: SELECT * FROM v_epa_predictions WHERE prediction_run_id = '619d8e03-212c-4145-a6bd-71439df9f28f'
  2. Wide format view: SELECT * FROM v_epa_measurements_wide_prediction WHERE prediction_run_id = '619d8e03-212c-4145-a6bd-71439df9f28f'
  3. Integrate with backend API to serve predictions to frontend
